<a href="https://colab.research.google.com/github/kkorhone/Python_Notebooks/blob/main/NusseltNumber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Nusselt number estimation**

In [133]:
import numpy

k_fluid = 0.407
Cp_fluid = 4206
rho_fluid = 967
mu_fluid = 6.495e-3
Q_fluid = 0.6e-3

r_outer = 20e-3
r_inner = 17.6e-3
r_borehole = 140e-3
L_borehole = 300

Dh = 2 * r_inner
A_inner = numpy.pi * r_inner**2

v_fluid = Q_fluid / A_inner
Pr_fluid = Cp_fluid * mu_fluid / k_fluid
Re_fluid = rho_fluid * v_fluid * Dh / mu_fluid

print(f"v_fluid = {v_fluid}")
print(f"Pr_fluid = {Pr_fluid}")
print(f"Re_fluid = {Re_fluid}")

v_fluid = 0.6165609882175697
Pr_fluid = 67.12031941031941
Re_fluid = 3231.2162496297037


# Darcy friction factor

**1. Colebrook-White equation**

$\frac{1}{\sqrt{f}} = -2\cdot\log\left(\frac{\varepsilon/D}{3.7}+\frac{2.51}{Re\cdot\sqrt{f}}\right)$

**2. Haaland equation**

$\frac{1}{\sqrt{f}} = -1.8\cdot\log\left[\left(\frac{\varepsilon/D}{3.7}\right)^{1.11}+\frac{6.9}{Re}\right]$

**Solving for f gives**

$f = \frac{1}{\left\{-1.8\cdot\log\left[\left(\frac{\varepsilon/D}{3.7}\right)^{1.11}+\frac{6.9}{Re}\right]\right\}^2}$

**3. Swamee–Jain equation**

$f = \frac{0.25}{\left[\log\left(\frac{\varepsilon/D}{3.7}+\frac{5.74}{Re^{0.9}}\right)\right]^2}$

**4. Moody equation**

$f = 0.0055\cdot\left[1+\left(2\cdot{10^4}\cdot\frac{\varepsilon}{D}+\frac{10^6}{Re}\right)^{1/3}\right]$

$\varepsilon$ = pipe roughness (for PE it is $0.0015-0.007\cdot{10^{-3}}$ m)

In [134]:
import scipy.optimize

epsilon = 0.001e-3

left = lambda f: 1 / numpy.sqrt(f)
rite = lambda f: -2 * numpy.log10(epsilon / Dh / 3.7 + 2.51 / (Re_fluid * numpy.sqrt(f)))

f = scipy.optimize.fminbound(lambda f: abs(left(f)-rite(f)), 1e-6, 10)

print(f"Colebrook-White: f = {f}")

f = 1 / (-1.8 * numpy.log10((epsilon/Dh/3.7)**1.11+6.9/Re_fluid))**2

print(f"Haaland:         f = {f}")

f = 0.25 / (numpy.log10(epsilon/Dh/3.7+5.74/Re_fluid**0.9))**2

print(f"Swamee-Jain:     f = {f}")

f = 0.0055 * (1 + (2e4 * epsilon / Dh + 1e6 / Re_fluid)**(1/3))

print(f"Moody:           f = {f}")

Colebrook-White: f = 0.04257124892451207
Haaland:         f = 0.043291566880123054
Swamee-Jain:     f = 0.043450500182242204
Moody:           f = 0.042725414947420964


# Correlations for Nusselt number

**1. FEFLOW White Paper**

$\xi = \left(1.8\cdot\log{Re}-1.5\right)^{-2}$

$\gamma = \frac{Re-2300}{10^4-2300}$

$Nu =
\begin{cases}
  4.364 & \text{for laminar flow if $Re<2300$} \\
  (1-\gamma)\cdot{4.364}+\gamma\cdot\left\{\frac{(0.0308/8)\cdot{10^4}\cdot{Pr}}{1+12.7\cdot\sqrt{0.0308/8}\cdot(Pr^{2/3}-1)}\cdot\left[1+\left(\frac{D_h}{L}\right)^{2/3}\right]\right\} & \text{for transitional flow if $2300\le Re<10^4$} \\
  \frac{(\xi/8)\cdot{Re}\cdot{Pr}}{1+12.7\cdot\sqrt{\xi/8}\cdot{(Pr^{2/3}-1)}}\cdot\left[1+\left(\frac{D_h}{L}\right)^{2/3}\right] & \text{for turbulent flow if $Re\ge{10^4}$}
\end{cases}$

**2. Dittus-Boelter**

$Nu = 0.023\cdot{Re^{0.8}}\cdot{Pr^n}$

$n$ is $0.4$ when the fluid is being heated and $0.3$ when cooled

**3. Wikipedia**

$Nu = \frac{(f/8)\cdot(Re-1000)\cdot{Pr}}{1+12.7\cdot(f/8)^{1/2}\cdot(Pr^{2/3}-1)}$

is valid for $2300 < Re < 10^6$ and $0.6 < Pr < 10^5$

In [140]:
xi = (1.8 * numpy.log10(Re_fluid) - 1.5)**(-2)
gamma = (Re_fluid - 2300) / (1e4 - 2300)

print(f"Friction factor:     xi = {xi}")

if Re_fluid < 2300:
  print(f"Laminar flow:        {Re_fluid} < 2300")
elif Re_fluid < 1e4:
  print(f"Transitional flow:   2300 <= {Re_fluid} < 10000")
else:
  print(f"Turbulent flow:      {Re_fluid} >= 10000")

if gamma < 0:
  Nu_fluid = 4.364
elif gamma < 1:
  Nu_fluid = (1-gamma)*4.364+gamma*((0.0308/8)*1e4*Pr_fluid/(1+12.7*numpy.sqrt(0.0308/8)*(Pr_fluid**(2/3)-1))*(1+(Dh/L_borehole)**(2/3)))
else:
  Nu_fluid = (xi/8)*Re_fluid*Pr_fluid/(1+12.7*numpy.sqrt(xi/8)*(Pr_fluid**(2/3)-1))*(1+(Dh/L_borehole)**(2/3))

print(f"FEFLOW White Paper:  Nu_fluid = {Nu_fluid}")

Nu_fluid = 0.023 * Re_fluid**0.8 * Pr_fluid**0.4

print(f"Dittus-Boelter:      Nu_fluid = {Nu_fluid}")

f = (0.79 * numpy.log(Re_fluid) - 1.64)**(-2)

Nu_fluid = (f/8)*(Re_fluid-1000)*Pr_fluid/(1+12.7*numpy.sqrt(f/8)*(Pr_fluid**(2/3)-1))

print(f"Wikipedia:           Nu_fluid = {Nu_fluid}")

Friction factor:     xi = 0.043099492933196384
Transitional flow:   2300 <= 3231.2162496297037 < 10000
FEFLOW White Paper:  Nu_fluid = 27.520532765235863
Dittus-Boelter:      Nu_fluid = 79.42592131462405
Wikipedia:           Nu_fluid = 53.03298550645952
